# GONZAG WestMed example

This notebook demonstrates how you can blablabla

## Loading & initilizations

In [26]:
import sys
from os import getenv

GONZAG_DIR   = getenv('HOME')+'/DEV/gonzag'          ; print(GONZAG_DIR)
CLIMPORN_DIR = getenv('HOME')+'/DEV/climporn/python' ; print(CLIMPORN_DIR)

sys.path.append(GONZAG_DIR)

import gonzag as gz
from gonzag.config import ldebug

GONZAG_DATA_DIR = '/MEDIA/data/GONZAG/gonzag_input'

# Satellite input data:
file_sat = GONZAG_DATA_DIR+'/SENTINEL3A_20170130-20170303.nc'
name_ssh_sat = 'sla_unfiltered'

# Model input data:
file_mod = GONZAG_DATA_DIR+'/sossheig_box_WestMed_eNATL60-BLBT02_20170201-20170228.nc'
name_ssh_mod = 'sossheig'
file_lsm_mod = file_mod; name_lsm_mod = '_FillValue' ; # we use _FillValue attribute of "nams_ssh_mod" in "file_mod"
l_griddist = False ; # grid is not strongly distorded





/home/laurent/DEV/gonzag
/home/laurent/DEV/climporn/python


### Time overlap between model and satellite data ?

In [27]:
    (it1,it2), (Nts,Ntm) = gz.GetEpochTimeOverlap( file_sat , file_mod )
    print(' *** Time overlap between model and satellite in UNIX epoch time: it1, it2', it1,'--',it2)
    print('   => UTC: "'+gz.EpochT2Str(it1)+'" to "'+gz.EpochT2Str(it2)+'"\n')


 *** [GetTimeInfo()] Getting calendar/time info in /MEDIA/data/GONZAG/gonzag_input/SENTINEL3A_20170130-20170303.nc ...
   => time/record dimension is "time"
 *** [ToEpochTime()]: original t0 as "days since 1950-01-01 00:00:00" =>  24500.98934059627
 *** [ToEpochTime()]: intitial date in datetime format =>  2017-01-29 23:44:39.27517
 *** [ToEpochTime()]: original t0 as "days since 1950-01-01 00:00:00" =>  24534.993929587326
 *** [ToEpochTime()]: intitial date in datetime format =>  2017-03-04 23:51:15.516345
   => first and last time records:  2017-01-29 23:44:39.27517 -- 2017-03-04 23:51:15.516345  (UNIX epoch:  1485733479.027517 -- 1488671475.516345 )

 *** [GetTimeInfo()] Getting calendar/time info in /MEDIA/data/GONZAG/gonzag_input/sossheig_box_WestMed_eNATL60-BLBT02_20170201-20170228.nc ...
   => time/record dimension is "time_counter"
 *** [ToEpochTime()]: original t0 as "seconds since 1900-1-1 00:00:00" =>  3694897800.0
 *** [ToEpochTime()]: intitial date in datetime format =>  2

## Create object `ModelGrid` containing all the model (aka _source_) 2D+T domain grid info

In [28]:
    clsm = name_lsm_mod
    if name_lsm_mod=='_FillValue': clsm = name_lsm_mod+'@'+name_ssh_mod
    
    ModelGrid = gz.ModGrid( file_mod, it1, it2, file_lsm_mod, clsm, distorded_grid=l_griddist )


 *** [GetTimeEpochVector()] reading "time_counter" in /MEDIA/data/GONZAG/gonzag_input/sossheig_box_WestMed_eNATL60-BLBT02_20170201-20170228.nc and converting it to Epoch time...
 *** [ToEpochTime()]: original t0 as "seconds since 1900-1-1 00:00:00" =>  3694897800.0
 *** [ToEpochTime()]: intitial date in datetime format =>  2017-02-01 00:30:00
   => 672 records in TOTAL!

 *** [GetTimeEpochVector()] reading "time_counter" in /MEDIA/data/GONZAG/gonzag_input/sossheig_box_WestMed_eNATL60-BLBT02_20170201-20170228.nc and converting it to Epoch time...
 *** [ToEpochTime()]: original t0 as "seconds since 1900-1-1 00:00:00" =>  3694897800.0
 *** [ToEpochTime()]: intitial date in datetime format =>  2017-02-01 00:30:00
   => 672 records in TOTAL!

 *** [GetModelCoor()] Read model latitude (variable is "gphit", with 2 dimensions! (796, 868) 

 *** [GetModelCoor()] Read model longitude (variable is "glamt", with 2 dimensions! (796, 868) 


 *** what we use to define model land-sea mask:
    => "_F

## Create object `SatelliteTrack` containing all the satellite track (aka _target_) info

In [29]:
SatelliteTrack = gz.SatTrack( file_sat, it1, it2, Np=Nts, \
                              domain_bounds=ModelGrid.domain_bounds, l_0_360=ModelGrid.l360 )

 *** [SatTrack()] Analyzing the time vector in /MEDIA/data/GONZAG/gonzag_input/SENTINEL3A_20170130-20170303.nc ...
 *** [ToEpochTime()]: original t0 as "days since 1950-01-01 00:00:00" =>  24500.98934059627
 *** [ToEpochTime()]: intitial date in datetime format =>  2017-01-29 23:44:39.27517
 *** [ToEpochTime()]: original t0 as "days since 1950-01-01 00:00:00" =>  24503.013806444927
 *** [ToEpochTime()]: intitial date in datetime format =>  2017-02-01 00:19:52.876841
 *** [GetTimeEpochVector()] reading "time" in /MEDIA/data/GONZAG/gonzag_input/SENTINEL3A_20170130-20170303.nc and converting it to Epoch time...
 *** [ToEpochTime()]: original t0 as "days since 1950-01-01 00:00:00" =>  24503.01463977826
 *** [ToEpochTime()]: intitial date in datetime format =>  2017-02-01 00:21:04.876841
   => 1315362 records read...

 *** ROOM FOR IMPROVEMENT => time_read_time = 0.9166145324707031 

 *** [GetSatCoor()] reading "latitude" in /MEDIA/data/GONZAG/gonzag_input/SENTINEL3A_20170130-20170303.nc ..

In [30]:
ii = gz.Model2SatTrack( ModelGrid, name_ssh_mod, SatelliteTrack, name_ssh_sat, file_out='result.nc' )

 *** "found" distance criterion when searching for nearest point on model grid is  1.3961514174799983  km

 *** Will use zoom boxes of width of 269 points for 1st attempts of nearest-point location...


 *** Finding nearest points on source (model) grid... (rd_found_km, np_box_r = 1.3961514174799983 134 )
          => last tested distance criterions = 3.4085727965820274  km

          => last tested distance criterions = 3.4085727965820274  km

          => last tested distance criterions = 3.4085727965820274  km

          => last tested distance criterions = 3.4085727965820274  km

          => last tested distance criterions = 3.4085727965820274  km

          => last tested distance criterions = 3.4085727965820274  km

          => last tested distance criterions = 3.4085727965820274  km

          => last tested distance criterions = 3.4085727965820274  km

          => last tested distance criterions = 3.4085727965820274  km

          => last tested distance criterions = 3.40857

          => last tested distance criterions = 3.4085727965820274  km

          => last tested distance criterions = 3.4085727965820274  km

          => last tested distance criterions = 3.4085727965820274  km

          => last tested distance criterions = 3.4085727965820274  km

          => last tested distance criterions = 3.4085727965820274  km

          => last tested distance criterions = 3.4085727965820274  km

      +++ Treated point: 1500/2452 
          ==> Sat. coordinates:     35.716 -3.454
          ==> Model nearest point:  35.712 356.55  ( 48 122 )
          => last tested distance criterions = 3.4085727965820274  km

          => last tested distance criterions = 3.4085727965820274  km

          => last tested distance criterions = 3.4085727965820274  km

          => last tested distance criterions = 2.726858237265622  km

          => last tested distance criterions = 3.4085727965820274  km

          => last tested distance criterions = 3.4085727965820274  km

 

/home/laurent/DEV/gonzag/gonzag/bilin_mapping.py:108: RuntimeWarning: invalid value encountered in double_scalars
  zdalp = nmp.linalg.det( zM ) / zdeta
/home/laurent/DEV/gonzag/gonzag/bilin_mapping.py:111: RuntimeWarning: invalid value encountered in double_scalars
  zdbet = nmp.linalg.det( zM ) / zdeta


     ***    Done! *** 


 *** ploting meshes...
   ==> plot for jt = 0  /  2452
   ==> plot for jt = 500  /  2452
   ==> plot for jt = 1000  /  2452
   ==> plot for jt = 1500  /  2452
   ==> plot for jt = 2000  /  2452

 *** Satelite time at jt = 00000 ==>  Wed Feb  1 09:56:04 2017 1485942964.0724182
   => surounding kt for model:  9 10 ( Wed Feb  1 09:30:00 2017 Wed Feb  1 10:30:00 2017 ) /  1485941400.0 1485945000.0
 *** Reading sossheig in /MEDIA/data/GONZAG/gonzag_input/sossheig_box_WestMed_eNATL60-BLBT02_20170201-20170228.nc
    => at ktm1= 9
 *** [GetModel2DVar()] Reading model "sossheig" at record kt=9 in /MEDIA/data/GONZAG/gonzag_input/sossheig_box_WestMed_eNATL60-BLBT02_20170201-20170228.nc

 *** Reading sossheig in /MEDIA/data/GONZAG/gonzag_input/sossheig_box_WestMed_eNATL60-BLBT02_20170201-20170228.nc
    => at ktm2= 10
 *** [GetModel2DVar()] Reading model "sossheig" at record kt=10 in /MEDIA/data/GONZAG/gonzag_input/sossheig_box_WestMed_eNATL60-BLBT02_20170201-20170228.nc



 *** Reading sossheig in /MEDIA/data/GONZAG/gonzag_input/sossheig_box_WestMed_eNATL60-BLBT02_20170201-20170228.nc
    => at ktm1= 250
 *** [GetModel2DVar()] Reading model "sossheig" at record kt=250 in /MEDIA/data/GONZAG/gonzag_input/sossheig_box_WestMed_eNATL60-BLBT02_20170201-20170228.nc

 *** Reading sossheig in /MEDIA/data/GONZAG/gonzag_input/sossheig_box_WestMed_eNATL60-BLBT02_20170201-20170228.nc
    => at ktm2= 251
 *** [GetModel2DVar()] Reading model "sossheig" at record kt=251 in /MEDIA/data/GONZAG/gonzag_input/sossheig_box_WestMed_eNATL60-BLBT02_20170201-20170228.nc

 *** Reading sossheig in /MEDIA/data/GONZAG/gonzag_input/sossheig_box_WestMed_eNATL60-BLBT02_20170201-20170228.nc
    => at ktm1= 273
 *** [GetModel2DVar()] Reading model "sossheig" at record kt=273 in /MEDIA/data/GONZAG/gonzag_input/sossheig_box_WestMed_eNATL60-BLBT02_20170201-20170228.nc

 *** Reading sossheig in /MEDIA/data/GONZAG/gonzag_input/sossheig_box_WestMed_eNATL60-BLBT02_20170201-20170228.nc
    => at 

 *** Reading sossheig in /MEDIA/data/GONZAG/gonzag_input/sossheig_box_WestMed_eNATL60-BLBT02_20170201-20170228.nc
    => at ktm1= 500
 *** [GetModel2DVar()] Reading model "sossheig" at record kt=500 in /MEDIA/data/GONZAG/gonzag_input/sossheig_box_WestMed_eNATL60-BLBT02_20170201-20170228.nc

 *** Reading sossheig in /MEDIA/data/GONZAG/gonzag_input/sossheig_box_WestMed_eNATL60-BLBT02_20170201-20170228.nc
    => at ktm2= 501
 *** [GetModel2DVar()] Reading model "sossheig" at record kt=501 in /MEDIA/data/GONZAG/gonzag_input/sossheig_box_WestMed_eNATL60-BLBT02_20170201-20170228.nc

 *** Reading sossheig in /MEDIA/data/GONZAG/gonzag_input/sossheig_box_WestMed_eNATL60-BLBT02_20170201-20170228.nc
    => at ktm1= 514
 *** [GetModel2DVar()] Reading model "sossheig" at record kt=514 in /MEDIA/data/GONZAG/gonzag_input/sossheig_box_WestMed_eNATL60-BLBT02_20170201-20170228.nc

 *** Reading sossheig in /MEDIA/data/GONZAG/gonzag_input/sossheig_box_WestMed_eNATL60-BLBT02_20170201-20170228.nc
    => at 